In [4]:
import spacy

In [5]:
!python -m spacy download en_core_web_sm

     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     --------------------------------------- 0.0/12.8 MB 281.8 kB/s eta 0:00:46
     --------------------------------------- 0.1/12.8 MB 722.1 kB/s eta 0:00:18
     -- ------------------------------------- 0.7/12.8 MB 3.6 MB/s eta 0:00:04
     ------- -------------------------------- 2.4/12.8 MB 9.7 MB/s eta 0:00:02
     ------------- -------------------------- 4.2/12.8 MB 14.1 MB/s eta 0:00:01
     ------------------ --------------------- 5.9/12.8 MB 17.2 MB/s eta 0:00:01
     ----------------------- ---------------- 7.6/12.8 MB 19.5 MB/s eta 0:00:01
     ----------------------------- ---------- 9.4/12.8 MB 21.6 MB/s eta 0:00:01
     ---------------------------------- ---- 11.2/12.8 MB 38.6 MB/s eta 0:00:01
     --------------------------------------  12.8/12.8 MB 36.4 MB/

In [ ]:
nlp = spacy.load("en_core_web_sm")
nlp

# Custom NER for Cooking texts

In [2]:
import json
with open('TASTEset.json', 'r') as f:
    data = json.load(f)

In [46]:
data[0][0]


'5 ounces rum\n4 ounces triple sec\n3 ounces Tia Maria\n20 ounces orange juice\n'

In [3]:
data[1][1]["entities"][8]

[62, 68, 'FOOD']

In [58]:
data[1][1]["entities"][1][2]

'UNIT'

In [4]:
# Initialiser une liste vide pour stocker les données d'entraînement
training_data = []

# Itérer sur chaque exemple dans les données fournies
for example in data:
    # Créer un dictionnaire temporaire pour contenir le texte et les entités de chaque exemple
    temp_dict = {
        'text': example[0],  # Stocker le texte de l'exemple
        'entities': []  # Initialiser une liste vide pour stocker les entités
    }

    # Itérer sur chaque annotation dans l'exemple actuel
    for annotation in example[1]['entities']:
        # Extraire les positions de début et de fin de l'entité, ainsi que son libellé
        start = annotation[0]
        end = annotation[1]
        label = annotation[2].upper()  # Convertir le libellé en majuscules

        # Ajouter l'information de l'entité sous forme de tuple à la liste des entités
        temp_dict['entities'].append((start, end, label))

    # Ajouter le dictionnaire préparé à la liste des données d'entraînement
    training_data.append(temp_dict)

# Afficher la première entrée des données d'entraînement formatées pour vérifier la structure
print(training_data[0])


{'text': '5 ounces rum\n4 ounces triple sec\n3 ounces Tia Maria\n20 ounces orange juice\n', 'entities': [(0, 1, 'QUANTITY'), (2, 8, 'UNIT'), (9, 12, 'FOOD'), (13, 14, 'QUANTITY'), (15, 21, 'UNIT'), (22, 32, 'FOOD'), (33, 34, 'QUANTITY'), (35, 41, 'UNIT'), (42, 51, 'FOOD'), (52, 54, 'QUANTITY'), (55, 61, 'UNIT'), (62, 74, 'FOOD')]}


In [5]:
training_data[11]['text']

'1 ounce Kahlua\n1⁄2 ounce tequila\n6 ounces hot coffee\nwhipped cream\n'

In [6]:
training_data[11]['entities']

[(0, 1, 'QUANTITY'),
 (2, 7, 'UNIT'),
 (8, 14, 'FOOD'),
 (15, 18, 'QUANTITY'),
 (19, 24, 'UNIT'),
 (25, 32, 'FOOD'),
 (33, 34, 'QUANTITY'),
 (35, 41, 'UNIT'),
 (42, 45, 'PHYSICAL_QUALITY'),
 (46, 52, 'FOOD'),
 (53, 60, 'PROCESS'),
 (61, 66, 'FOOD')]

In [116]:
# Import necessary functions from spaCy
from spacy.util import filter_spans
from tqdm import tqdm

# Initialize the spaCy model
nlp = spacy.blank("en") # Create a blank English model

# Initialize the DocBin object for efficient storage of Doc objects
doc_bin = DocBin()

# Process each training example
for training_example in tqdm(training_data):
    # Extract text and entity labels from the training example
    text = training_example['text']
    labels = training_example['entities']

    # Create a Doc object from the text without running the full pipeline
    doc = nlp.make_doc(text)

    # Initialize an empty list to store entity spans
    ents = []

    # Iterate over the entities in the training example
    for start, end, label in labels:
        # Create a span for each entity
        span = doc.char_span(start, end, label=label, alignment_mode="contract")

        # If the span is valid, add it to the list of entities
        if span is None:
            print("Skipping entity")
        else:
            ents.append(span)

    # Filter overlapping spans to ensure each token is part of at most one entity
    filtered_ents = filter_spans(ents)

    # Assign the filtered entities to the document
    doc.ents = filtered_ents

    # Add the processed document to the DocBin
    doc_bin.add(doc)

# Save the DocBin with the processed documents to disk
doc_bin.to_disk("train.spacy")


100%|██████████| 700/700 [00:00<00:00, 1798.10it/s]


In [117]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [122]:
import os
os.environ['KMP_DUPLICATE_LIB_OK'] = 'TRUE'


In [ ]:
!conda install -c conda-forge libopenblas


En apprentissage automatique, max_epochs revient à décider combien de fois SpaCy lira le « livre d'histoires » (les données d'entraînement).
Chaque « époque » est une lecture complète du livre de contes du début à la fin. Ainsi, max_epochs indique à SpaCy combien de fois lire toutes les données d'entraînement.

Pour ce didacticiel, définissez la taille du lot sur 100 et max_epochs dans [training] sur 30. --- la valeur par défaut est 0 : une valeur par défaut de 0 signifie souvent que l'entraînement se poursuivra jusqu'à ce qu'une autre condition d'arrêt soit remplie. Ces conditions peuvent être des choses comme le fait de ne pas beaucoup mieux comprendre l'histoire (le modèle ne s'améliore pas), ou cela peut être basé sur d'autres paramètres ou critères.

In [ ]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy

**Interprétation des résultats de l'entraînement :**


L'interprétation de la sortie d'un modèle personnalisé de reconnaissance d'entités nommées (NER) dans spaCy implique de comprendre diverses métriques et valeurs qui sont généralement affichées pendant le processus de formation. Décomposons chaque partie du résultat que vous avez fourni, qui semble provenir du journal d'entraînement :

1. **E** : Cela signifie « Époque ». Une époque est un passage complet à travers l’ensemble des données d’entraînement. Si vous voyez « E 5 », cela signifie que SpaCy en est à son 5ème cycle de lecture de tous ses livres (données d'entraînement).

2. **#** : Il s'agit du numéro de lot. À chaque époque, les données d'entraînement sont divisées en ensembles plus petits ou « lots ». Ce numéro indique de quel lot SpaCy apprend actuellement à l'époque.

3. **LOSS TOK2VEC** : Il s'agit de la perte d'entraînement pour le composant "tok2vec" du modèle. La perte mesure la capacité de SpaCy à comprendre les mots de son lot actuel. Une perte inférieure signifie que tout va mieux. S'il est trop élevé, SpaCy pourrait avoir des problèmes.

4. **PERTE NER** : Il s'agit de la perte d'entraînement spécifiquement pour la partie NER. Il vous indique dans quelle mesure SpaCy réussit à trouver et à comprendre les noms ou choses (entités) spéciaux dans le texte. Comme avec tok2vec, un nombre inférieur est préférable.

5. **ENTS_F** : Il s'agit du score F pour la reconnaissance d'entité. C'est un nombre compris entre 0 et 1 qui mesure la précision de SpaCy pour trouver les bonnes entités. C'est un mélange de précision et de rappel (qui sont expliqués ci-dessous). Plus proche de 1 signifie que SpaCy est vraiment bon dans ce domaine !

6. **ENTS_P** : Il s'agit du score de précision des entités. Il vous indique quel pourcentage des entités trouvées par SpaCy étaient réellement correctes. Si SpaCy est trop impatient et marque trop de choses comme entités, ce nombre pourrait diminuer.

7. **ENTS_R** : Il s'agit du score de rappel des entités. Il mesure le pourcentage d'entités réelles dans le texte que SpaCy a réussi à trouver. Si SpaCy est trop prudent et manque certaines entités, ce nombre pourrait être faible.

8. **SCORE** : Il s'agit généralement du score global du modèle, souvent basé sur le score F. Cela vous donne une idée rapide des performances du modèle. Plus haut, c'est mieux.

Lorsque vous entraînez SpaCy, vous voulez voir le LOSS TOK2VEC et le LOSS NER diminuer au fil du temps à mesure qu'il apprend. Dans le même temps, vous souhaitez que ENTS_F, ENTS_P et ENTS_R augmentent, montrant que SpaCy s'améliore dans la recherche et la compréhension des bonnes entités dans le texte. C'est comme regarder SpaCy devenir plus intelligent et meilleur dans sa lecture à chaque époque et à chaque lot !


In [14]:
# Import necessary functions from spaCy
from spacy.util import filter_spans
from tqdm import tqdm

# Initialize the spaCy model
nlp = spacy.blank("en") # Create a blank English model

# Initialize the DocBin object for efficient storage of Doc objects
doc_bin = DocBin()


NameError: name 'DocBin' is not defined

In [11]:
import spacy
nlp_ner = spacy.load("model-best")

[(0, 1, 'QUANTITY'),
 (2, 7, 'UNIT'),
 (8, 14, 'FOOD'),
 (15, 18, 'QUANTITY'),
 (19, 24, 'UNIT'),
 (25, 32, 'FOOD'),
 (33, 34, 'QUANTITY'),
 (35, 41, 'UNIT'),
 (42, 45, 'PHYSICAL_QUALITY'),
 (46, 52, 'FOOD'),
 (53, 60, 'PROCESS'),
 (61, 66, 'FOOD')]

In [26]:
# Intitalize the colours for the NER

colors = {
    "FOOD": "#F67DE3",
    "QUANTITY": "#7DF6D9",
    "PHYSICAL_QUALITY": "#a6e22d",
    "UNIT": "#6D7DF6",
    "PROCESS": "#F6807D"
}
options = {"colors": colors}

In [27]:
doc = nlp_ner("In the world of culinary arts, the quest for the perfect balance of quantity and quality is an eternal pursuit. Whether it's a hearty stew simmering on a cold winter's day or a delicate dessert adorned with intricate decorations, chefs aim to strike the ideal harmony between the amount of ingredients used and the exceptional quality of the final dish. The essence of culinary excellence lies not only in the flavors and textures but also in the presentation and aesthetics. Every bite should be a delightful experience, a symphony of taste and texture that tantalizes the senses and leaves a lasting impression.")

spacy.displacy.render(doc, style="ent", options= options, jupyter=True)

In [28]:
doc = nlp_ner("As the sun sets on a warm summer evening, the aroma of freshly grilled food fills the air, enticing everyone with its irresistible allure. Whether it's succulent steaks sizzling on the barbecue or colorful vegetables charred to perfection, the joy of outdoor cooking lies in the process of creating memorable meals that bring people together.")
spacy.displacy.render(doc, style="ent", options= options, jupyter=True)